In [0]:
from neuralprophet import NeuralProphet, set_log_level
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf
import warnings
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
import plotly.graph_objects as go
from copy import deepcopy
warnings.filterwarnings("ignore", category=FutureWarning)
set_log_level("ERROR")

Model Eval

In [0]:
def model_testing (test_df, test_forecast):
  mse=np.sqrt(mean_squared_error(y_true=test_df['y'], y_pred=test_forecast['yhat1']))
  mae=mean_absolute_error(y_true=test_df['y'], y_pred=test_forecast['yhat1'])
  mape=mean_absolute_percentage_error(y_true=test_df['y'], y_pred=test_forecast['yhat1'])

  print('Mean Squared Error:', mse)
  print('Mean Absolute Error:', mae)
  print('Mean Absolute Percentage Error:', mape)

Import dataframe

In [0]:
df = pd.read_csv('/Workspace/Users/ryan@delve.systems/Prophet_AI/Wellness_Sales_Grouped.csv')

df.rename(columns = {'trandate':'ds', 'AvgSale':'y'}, inplace = True)
df.head()

In [0]:


fig = go.Figure()

# Add actual data
fig.add_trace(go.Scatter(x=df['ds'], y=df['y'], mode='lines', name='Actual'))

# Update layout
fig.update_layout(
    title='Actuals',
    xaxis_title='Date',
    yaxis_title='Value'
)

# Show the plot
fig.show()

Create Train/Test Split

In [0]:
# Make sure ds is datetime
df['ds'] = pd.to_datetime(df['ds'])

# Split by date
split_date = df['ds'].max() - pd.Timedelta(days=90)

train_df = df[df['ds'] < split_date]
test_df = df[df['ds'] >= split_date]

Build and train base model

In [0]:
m_base = NeuralProphet(
    yearly_seasonality=True,
    weekly_seasonality=True,
    daily_seasonality=False,
)

metrics_base = m_base.fit(train_df)
base_forecast = m_base.predict(test_df)
base_forecast.head()

In [0]:
fig, ax = plt.subplots(figsize=(10, 6))

# Plotting the test_df (actual values)
ax.plot(test_df['ds'], test_df['y'], label='Actual', color='blue')

# Plotting the base_forecast (predicted values)
ax.plot(base_forecast['ds'], base_forecast['yhat1'], label='Forecast', color='red')
# Adding labels and title
ax.set_xlabel('Date')
ax.set_ylabel('Value')
ax.set_title('Actual vs. Forecasted Values')
ax.legend()

# Display the plot
plt.show()

In [0]:
model_testing(test_df,base_forecast)

Build and train model with holidays

In [0]:
holidays_df = pd.DataFrame({
    "event": 'Holiday',
    "ds": pd.to_datetime([
        "2023-03-21",
        "2023-04-20",
        "2023-04-21",
        "2023-04-27",
        "2023-05-01",
        "2023-06-28",
        "2023-06-29",
        "2023-09-25",
        "2023-12-01",
        "2023-12-16",
        "2023-12-25",
        "2023-12-26",
        "2024-01-01",
        "2024-03-21",
        "2024-03-29",
        "2024-04-01",
        "2024-04-27",
        "2024-05-01",
        "2024-06-16",
        "2024-06-17",
        "2024-08-09",
        "2024-09-24",
        "2024-12-16",
        "2024-12-25",
        "2024-12-26",
        "2025-01-01",  # New Year’s Day
        "2025-03-21",  # Human Rights Day
        "2025-04-18",  # Good Friday
        "2025-04-21",  # Family Day
        "2025-04-27",  # Freedom Day
        "2025-04-28",  # Freedom Day Observed
        "2025-05-01",  # Workers' Day
        "2025-06-16",  # Youth Day
        "2025-08-09",  # National Women’s Day
        "2025-09-24",  # Heritage Day
        "2025-12-16",  # Day of Reconciliation
        "2025-12-25",  # Christmas Day
        "2025-12-26"   # Day of Goodwill
    ]),
    "lower_window": 0,
    "upper_window": 0
})

holidays_df.head()



In [0]:
m_holidays = NeuralProphet(
    yearly_seasonality=True,
    weekly_seasonality=True,
    daily_seasonality=False,
    seasonality_mode='additive',
)

m_holidays.add_events('Holiday')

# Create a combined dataframe with events
df_with_holidays = m_holidays.create_df_with_events(train_df, holidays_df)

# Fit the model with the combined dataframe
metrics_holidays = m_holidays.fit(df_with_holidays)

# For prediction, you need to include events in the test period too
# First, filter holidays that fall within the test period
test_start = test_df['ds'].min()
test_end = test_df['ds'].max()
test_holidays = holidays_df[(holidays_df['ds'] >= test_start) & (holidays_df['ds'] <= test_end)]

# Create test dataframe with events
test_with_holidays = m_holidays.create_df_with_events(test_df, test_holidays)

# Make predictions
base_forecast_holidays = m_holidays.predict(test_with_holidays)
base_forecast_holidays.head()

In [0]:
fig, ax = plt.subplots(figsize=(10, 6))

# Plotting the test_df (actual values)
ax.plot(test_df['ds'], test_df['y'], label='Actual', color='blue')

# Plotting the base_forecast (predicted values)
ax.plot(base_forecast_holidays['ds'], base_forecast_holidays['yhat1'], label='Forecast', color='red')
# Adding labels and title
ax.set_xlabel('Date')
ax.set_ylabel('Value')
ax.set_title('Actual vs. Forecasted Values')
ax.legend()

# Display the plot
plt.show()

In [0]:
model_testing(test_df[0:89], base_forecast_holidays[0:89])

Build and train model with autoregression

In [0]:
holidays_df = pd.DataFrame({
    "event": 'Holiday',
    "ds": pd.to_datetime([
        "2023-03-21",
        "2023-04-20",
        "2023-04-21",
        "2023-04-27",
        "2023-05-01",
        "2023-06-28",
        "2023-06-29",
        "2023-09-25",
        "2023-12-01",
        "2023-12-16",
        "2023-12-25",
        "2023-12-26",
        "2024-01-01",
        "2024-03-21",
        "2024-03-29",
        "2024-04-01",
        "2024-04-27",
        "2024-05-01",
        "2024-06-16",
        "2024-06-17",
        "2024-08-09",
        "2024-09-24",
        "2024-12-16",
        "2024-12-25",
        "2024-12-26",
        "2025-01-01",  # New Year’s Day
        "2025-03-21",  # Human Rights Day
        "2025-04-18",  # Good Friday
        "2025-04-21",  # Family Day
        "2025-04-27",  # Freedom Day
        "2025-04-28",  # Freedom Day Observed
        "2025-05-01",  # Workers' Day
        "2025-06-16",  # Youth Day
        "2025-08-09",  # National Women’s Day
        "2025-09-24",  # Heritage Day
        "2025-12-16",  # Day of Reconciliation
        "2025-12-25",  # Christmas Day
        "2025-12-26"   # Day of Goodwill
    ]),
    "lower_window": 0,
    "upper_window": 0
})

In [0]:
# Create a NeuralProphet model with default parameters
params = {
    'n_lags': 60,
    'n_forecasts': 7,
    'ar_reg': 0.18351866767708624,
    'seasonality_mode': 'additive',
    'yearly_seasonality': True,
    'weekly_seasonality': True,
    'daily_seasonality': False,
}

m = NeuralProphet(params)
# Use static plotly in notebooks
m.set_plotting_backend("plotly-static")

m.add_events('Holiday')

# Create a combined dataframe with events
df_with_holidays = m.create_df_with_events(train_df, holidays_df)

# Fit the model with the combined dataframe
metrics = m.fit(df_with_holidays)

# For prediction, you need to include events in the test period too
# First, filter holidays that fall within the test period
test_start = test_df['ds'].min()
test_end = test_df['ds'].max()
test_holidays = holidays_df[(holidays_df['ds'] >= test_start) & (holidays_df['ds'] <= test_end)]

# Create test dataframe with events
test_with_holidays = m.create_df_with_events(test_df, test_holidays)

# Make predictions
forecast_holidays = m_holidays.predict(test_with_holidays)
forecast_holidays.head()

In [0]:
def recursive_predict(model, history_df, forecast_periods, n_forecasts=None):
    """
    Recursively extend a NeuralProphet forecast beyond the native forecast horizon.

    Args:
        model: Trained NeuralProphet model.
        history_df (pd.DataFrame): The historical training data with columns ['ds', 'y'].
        forecast_periods (int): Total number of days to forecast into the future.
        n_forecasts (int, optional): Number of steps forecasted per iteration.
                                    If None, uses model.n_forecasts.

    Returns:
        pd.DataFrame: DataFrame with dates and forecasted values.
    """
    # Use model's n_forecasts if not specified
    if n_forecasts is None:
        n_forecasts = params['n_forecasts']

    # Create a copy of history to avoid modifying the original
    df_history = deepcopy(history_df)

    # Create a dataframe to store all forecasts
    all_forecasts = pd.DataFrame(columns=['ds', 'yhat1'])

    # Calculate how many iterations we need
    remaining_periods = forecast_periods

    while remaining_periods > 0:
        print(f"Forecasting {min(n_forecasts, remaining_periods)} days ahead...")

        # Generate future dataframe
        periods = min(n_forecasts, remaining_periods)
        df_future = model.make_future_dataframe(
            df=df_history,
            periods=periods,
            n_historic_predictions=False
        )

        # Predict
        df_forecast = model.predict(df_future)

        if df_forecast is None or df_forecast.empty:
            print(f"Warning: Empty forecast")
            break

        # Check available forecast columns
        forecast_cols = [col for col in df_forecast.columns if col.startswith('yhat')]
        print(f"Available forecast columns: {forecast_cols}")

        # Add forecasts to our collection
        forecast_rows = []
        new_history_rows = []

        # If we only have yhat1 column but need to forecast multiple days
        if len(forecast_cols) == 1 and forecast_cols[0] == 'yhat1':
            # Use each row's yhat1 value
            for i in range(len(df_forecast)):
                forecast_row = {
                    'ds': df_forecast['ds'].iloc[i],
                    'yhat1': df_forecast['yhat1'].iloc[i]
                }
                forecast_rows.append(forecast_row)

                # Also add to history for next iteration
                new_history_row = {
                    'ds': df_forecast['ds'].iloc[i],
                    'y': df_forecast['yhat1'].iloc[i]
                }
                new_history_rows.append(new_history_row)
        else:
            # Handle multi-column case (yhat1, yhat2, etc.)
            for j in range(min(len(forecast_cols), len(df_forecast))):
                if j < len(df_forecast):
                    yhat_col = forecast_cols[j]
                    forecast_row = {
                        'ds': df_forecast['ds'].iloc[j],
                        'yhat1': df_forecast[yhat_col].iloc[j]
                    }
                    forecast_rows.append(forecast_row)

                    new_history_row = {
                        'ds': df_forecast['ds'].iloc[j],
                        'y': df_forecast[yhat_col].iloc[j]
                    }
                    new_history_rows.append(new_history_row)

        # Add to forecasts
        if forecast_rows:
            all_forecasts = pd.concat([all_forecasts, pd.DataFrame(forecast_rows)], ignore_index=True)

            # Add to history for next iteration
            if new_history_rows:
                df_append = pd.DataFrame(new_history_rows)
                df_history = pd.concat([df_history, df_append], ignore_index=True)

                # Update remaining periods
                remaining_periods -= len(new_history_rows)
            else:
                print(f"Warning: No valid forecasts generated")
                break
        else:
            print(f"Warning: No valid forecasts generated")
            break

    # Limit to requested forecast periods
    all_forecasts = all_forecasts.head(forecast_periods)

    if len(all_forecasts) == 0:
        print("No forecasts generated.")
        return None

    return all_forecasts

In [0]:
forecast_90day = recursive_predict(m, train_df, forecast_periods=90)
print(forecast_90day)

In [0]:
plt.figure(figsize=(10, 6))
plt.plot(forecast_90day['ds'], forecast_90day['yhat1'], label='Forecast', color='orange')
plt.plot(test_df['ds'], test_df['y'], label='Actual', color='blue')
plt.xlabel('Date')
plt.ylabel('Forecasted Value')
plt.title('90 Day Forecasted Timeline')
plt.legend()
plt.grid(True)
plt.tight_layout()



In [0]:
model_testing(test_df[0:89],forecast_90day[0:89])

Export the model

In [0]:
import mlflow.pyfunc
model_path = "/Workspace/Users/ryan@delve.systems/Prophet_AI"
mlflow.pyfunc.save_model(path=model_path, python_model=m)